In [2]:
from global_config import config
import pandas as pd
import numpy as np
import rpy2
import os


data_dir_mnps = config.get_property('geo_dir')
data_dir      = config.get_property('data_dir')
fb_ppl_data   = config.get_property('covid_fb')
results_dir   = config.get_property('results_dir')

In [3]:
rt_df = pd.read_csv( os.path.join(results_dir, 'bog_rt', 'rt_df_bog_confirmation.csv') )
rt_df

,Unnamed: 0,date,variable,strat,type,median,mean,sd,lower_90,lower_50,lower_20,upper_20,upper_50,upper_90
0,0,18732.0,R,NA_character_,estimate,1.894922,1.903549,0.187843,1.619383,1.769329,1.843829,1.939549,2.028880,2.219971
1,1,18733.0,R,NA_character_,estimate,1.432933,1.429162,0.235609,1.031743,1.266595,1.370040,1.493022,1.583354,1.808134
2,2,18734.0,R,NA_character_,estimate,1.053691,1.062235,0.319068,0.567436,0.834956,0.964154,1.140098,1.277382,1.602132
3,3,18735.0,R,NA_character_,estimate,0.757465,0.793695,0.352951,0.293705,0.531899,0.665135,0.847492,1.010946,1.438734
4,4,18736.0,R,NA_character_,estimate,0.541306,0.603268,0.356838,0.150432,0.342613,0.456720,0.625923,0.789564,1.286717
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
123,123,18749.0,reported_cases,NA_character_,forecast,100.000000,519.622500,3680.267456,5.000000,35.000000,72.000000,141.400000,249.000000,1230.200000
124,124,18750.0,reported_cases,NA_character_,forecast,82.500000,379.503000,1815.531481,4.000000,29.000000,56.000000,116.400000,217.000000,1137.650000
125,125,18751.0,reported_cases,NA_character_,forecast,49.000000,370.049000,3222.359455,3.000000,17.000000,34.000000,71.400000,141.000000,908.700000
126,126,18752.0,reported_cases,NA_character_,forecast,32.000000,585.437500,9111.033796,1.000000,10.750000,22.000000,48.000000,94.000000,695.450000
